In [2]:
import numpy as np
import pandas as pd

c:\Users\SUNIL\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [14]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer

In [3]:
df = pd.read_csv('train.csv', usecols=['Age', 'Fare', 'SibSp', 'Parch', 'Survived'])

In [6]:
df.dropna(inplace=True)

In [5]:
df.head()

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500


In [7]:
df['family'] = df['SibSp'] + df['Parch']

In [10]:
df.drop(['SibSp', 'Parch'], axis=1, inplace=True)

In [11]:
df.head()

,Survived,Age,Fare,family
0,0,22.0,7.2500,1
1,1,38.0,71.2833,1
2,1,26.0,7.9250,0
3,1,35.0,53.1000,1
4,0,35.0,8.0500,0


In [12]:
X = df.drop(['Survived'], axis=1)
y = df['Survived']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
X_train.head()

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0


### Without binarization

In [18]:
clf = DecisionTreeClassifier()
y_pred = clf.fit(X_train, y_train).predict(X_test)

In [19]:
accuracy_score(y_pred, y_test)

0.6363636363636364

In [20]:
np.mean(cross_val_score(clf, X, y, cv=10, scoring='accuracy'))

0.6428794992175274

### with binarizer

In [21]:
from sklearn.preprocessing import Binarizer

In [23]:
# travelling alone or not
trf = ColumnTransformer([
    ('bin', Binarizer(threshold=0, copy=False), ['family'])
], remainder='passthrough')

In [24]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [29]:
pd.DataFrame(X_train_trf, columns=['family', 'Age', 'Fare', ])

,family,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [26]:
y_pred2 = clf.fit(X_train_trf, y_train).predict(X_test_trf)
accuracy_score(y_pred2, y_test)

0.6293706293706294

In [27]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(clf, X_trf, y, cv=10, scoring='accuracy'))

0.6289906103286385

In [31]:
def binarizer(threshold):
    trf = ColumnTransformer([
        ('bin', Binarizer(threshold=threshold, copy=False), ['family'])
    ], remainder='passthrough')
    
    X_train_trf = trf.fit_transform(X_train)
    X_test_trf = trf.transform(X_test)
    
    y_pred = clf.fit(X_train_trf, y_train).predict(X_test_trf)
    print('Accuracy:', accuracy_score(y_pred, y_test))
    
    X_trf = trf.fit_transform(X)
    print('Cross-Val-score :', np.mean(cross_val_score(clf, X_trf, y, cv=10, scoring='accuracy')))

In [43]:
binarizer(2)

Accuracy: 0.6503496503496503
Cross-Val-score : 0.6288928012519561
